# **NETWORK INTRUSION DETECTION SYSTEM (NIDS)**
## **Fase 2: Advanced Preprocessing, Feature Engineering & Selection**

### **Obiettivo Strategico**

Questo notebook trasforma i dati grezzi in un dataset ottimizzato per il Machine Learning, eliminando il rumore e i bias topologici, e arricchendo l'informazione tramite Feature Engineering mirato.

### **Metodologia :**

1. **Sanitizzazione Anti-Bias**: Rimozione tassativa di feature topologiche. Catillo et al. (2022) dimostrano che mantenere questi campi porta a *Data-Induced Leakage*, dove il modello impara la rete specifica invece dell'attacco.

2. **Variance Threshold**: Rimozione di feature quasi-costanti che non contribuiscono alla discriminazione

3. **Multicollinearity Analysis**: Identificazione e rimozione di feature ridondanti con correlazione > 0.95. L'obiettivo è ridurre il rischio di overfitting e migliorare l'interpretabilità

4. **Multiple Feature Selection Methods**: Combinazione di metodi complementari:
   - **Filter-based**: Correlation, Mutual Information, ANOVA F-test
   - **Embedded**: Random Forest Feature Importance
   - **Consensus Ranking**: Aggregazione dei risultati per identificare le feature più robuste

5. **Strict Data Splitting**: Suddivisione Stratificata 70/15/15. Il Validation Set è isolato per scegliere il modello migliore, il Test Set per il Benchmark finale.

7. **No-Leakage Policy**: SMOTE e Scaling **NON** vengono applicati in questo notebook. Verranno integrati dinamicamente nelle pipeline di training nel notebook successivo per evitare la contaminazione del Validation Set.


---

## **1. SETUP AMBIENTE**

In [ ]:
# 1. SETUP AMBIENTE

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import gc
import json
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import (
    SelectFromModel,
    VarianceThreshold,
    f_classif,
    mutual_info_classif,
)
from scipy.stats import spearmanr

# Configurazione Plotting SOTA-Style
sns.set_style('whitegrid')
plt.rcParams['figure.dpi'] = 120
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)

# Variabili di Stato (Riproducibilità)
STATE_VARS = {
    'RANDOM_SEED': 42,
    'TEST_SIZE': 0.15,
    'VAL_SIZE_REL': 0.1765,  # 15% del totale, calcolato sul residuo dell'85%
    'CORR_THRESHOLD': 0.95,
    'RF_N_ESTIMATORS': 100,
    'N_FEATURES_FINAL': 20,  # Numero finale di feature da selezionare
}

# Path Management - CORRETTI
DATA_PATH = '../output'  # Qui sta cicids2017_cleaned.parquet
OUTPUT_PATH = '../output/processed_datasets'  # Output finale
INTERMEDIATE_PATH = '../output/feature_analysis'  # Analisi intermedie
IMG_PATH = '../output/images/preprocessing'  # Immagini

for p in [OUTPUT_PATH, INTERMEDIATE_PATH, IMG_PATH]:
    os.makedirs(p, exist_ok=True)

print(" Setup completato. Librerie caricate.")

## **2. Caricamento e Sanitizzazione Anti-Bias**

### **Fondamento Teorico: Data-Induced Leakage**

**Problema:** I sistemi NIDS tradizionali includono identificatori di rete che creano **dipendenze topologiche**.

**Soluzione:** Rimozione completa delle feature topologiche:
**Destination Port**: Identificatore di servizio specifico della rete di test

### **Safety Check: Leakage della Label**
Verifichiamo che `Label` e `LabelEncoded` **NON** entrino mai nel set di feature `X`.

In [ ]:
print("[1/5] Caricamento Dataset...")

try:
    df = pd.read_parquet(os.path.join(DATA_PATH, "cicids2017_cleaned.parquet"))
    print(f" Dataset caricato. Shape: {df.shape}")
except FileNotFoundError:
    raise FileNotFoundError("File Parquet non trovato! Assicurati di aver eseguito il Notebook 01.")

# --- SANITIZZAZIONE ---
identifiers = [
    'Destination Port',
    # Nota: Source/Destination IP non presenti nel dataset -> controllato features
]

# Feature Leakage: LabelEncoded non deve stare in X
leakage_cols = ['Label_Encoded']

cols_to_drop = [c for c in identifiers + leakage_cols if c in df.columns]

if cols_to_drop:
    df.drop(columns=cols_to_drop, inplace=True)
    print(f" Sanitizzazione: Rimossi {len(cols_to_drop)} identificatori e colonne leakage.")
    print(f"   rimossi: {cols_to_drop}")

print(f" Shape post-sanitizzazione: {df.shape}")

## **3. Feature Engineerimg**

### **Fondamento Teorico: Ratio Features per Attacchi Asimmetrici**

**Problema:** Gli attacchi moderni alterano la **simmetria bidirezionale** del traffico:
- **DoS/DDoS**: Flood unidirezionale → Molti pacchetti Forward, pochi/zero Backward
- **Port Scan**: Probing rapido → Pacchetti piccoli Forward, timeout Backward
- **Botnet C&C**: Comunicazione asimmetrica → Comandi piccoli, risposte grandi

Abbiamo introdotto **Ratio Features** per catturare queste asimmetrie:

### **Feature Ingegnerizzate:**

#### **1. PacketRatio = Fwd Packets / Bwd Packets**
- **Traffico Normale**: Ratio ≈ 1 (simmetria TCP handshake, richiesta/risposta HTTP)
- **DoS Attack**: Ratio >> 1 (flood unidirezionale, nessuna risposta)
- **Invarianza Temporale**: Non dipende dalla durata del flusso

#### **2. ByteRatio = Fwd Bytes / Bwd Bytes**
- **Traffico Normale**: Ratio variabile ma bilanciato
- **Data Exfiltration**: Ratio << 1 (comandi piccoli, dati estratti grandi)
- **C&C Communication**: Ratio asimmetrico caratteristico

### **Vantaggi:**
1. **Robustezza**: Invarianti rispetto alla scala temporale del flusso
2. **Generalizzazione**: Catturano pattern d'attacco, non caratteristiche di rete
3. **Interpretabilità**: Valori > 1 o < 1 hanno significato semantico diretto

In [ ]:
print("[2/5] Feature Engineering FLARE Ratios...")

# 1. Packet Ratio (Simmetria pacchetti)
df['PacketRatio'] = np.where(
    df['Total Backward Packets'] == 0,
    0,
    df['Total Fwd Packets'] / df['Total Backward Packets'],
)

# 2. Byte Ratio (Simmetria payload)
df['ByteRatio'] = np.where(
    df['Total Length of Bwd Packets'] == 0,
    0,
    df['Total Length of Fwd Packets'] / df['Total Length of Bwd Packets'],
)

# Gestione infiniti e NaN generati dalla divisione
df.replace([np.inf, -np.inf], 0, inplace=True)
df.fillna(0, inplace=True)

print(" Feature create: PacketRatio, ByteRatio")

# Separazione X e y
target_col = 'Label'
X = df.drop(columns=target_col)
y = df[target_col]

# SAFETY CHECK RIGOROSO
assert 'Label' not in X.columns, " ERRORE CRITICO: La Label è in X!"
assert 'LabelEncoded' not in X.columns, " ERRORE CRITICO: LabelEncoded è in X!"
print(" Safety Check Passato: Nessun target nelle feature.")

del df
gc.collect()

## **4. Stratified Data Splitting**

### **Fondamento Teorico: Train/Val/Test Isolation**

**Problema dello Sbilanciamento Estremo:**
CIC-IDS-2017 presenta classi rare:
- **Heartbleed**: 11 campioni (0.001%)
- **Infiltration**: 36 campioni (0.004%)
- **BENIGN**: 2,273,097 campioni (80%)

### **Split 70/15/15 Stratificato:**
1. **Train Set (70%)**: Addestramento + Feature Selection
   - Feature Selection calcolata **SOLO** su Train per evitare leakage
2. **Validation Set (15%)**: Valutazione dei modelli
   - NON toccare durante Feature Selection
3. **Test Set (15%)**: Benchmark finale
   - Completamente isolato fino alla valutazione finale
   - Simula deployment reale

### **Stratificazione (`stratify=y`):**
- Garantisce che classi rare (Heartbleed, Infiltration) siano presenti in tutti i set
- Mantiene distribuzione proporzionale: Train/Val/Test hanno stesse proporzioni di attacchi

In [ ]:
print("[3/5] Data Splitting Stratificato (70/15/15)...")

# Split 1: Train+Val vs Test
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y,
    test_size=STATE_VARS['TEST_SIZE'],
    stratify=y,
    random_state=STATE_VARS['RANDOM_SEED'],
)

# Split 2: Train vs Val
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp,
    test_size=STATE_VARS['VAL_SIZE_REL'],
    stratify=y_temp,
    random_state=STATE_VARS['RANDOM_SEED'],
)

print(f" Train Set: {X_train.shape[0]:,} samples")
print(f" Val Set: {X_val.shape[0]:,} samples")
print(f" Test Set: {X_test.shape[0]:,} samples")

del X, y, X_temp, y_temp
gc.collect()

## **5. Feature Selection Pipeline SOTA**

### **Fondamento Teorico: Ensemble Feature Selection**

**Problema:** Metodi singoli di Feature Selection hanno bias:
- **Correlation**: Cattura solo relazioni lineari
- **Mutual Information**: Computazionalmente costosa, può overfittare
- **Random Forest**: Bias verso feature ad alta cardinalità
- **ANOVA F-test**: Assume normalità

**Tripathi et al. (2024)** e **Agarwala et al. (2024)** propongono **Consensus Ranking**:

### **Strategia Multi-Method:**
1. **Filter Methods** (veloci, no training):
   - Spearman Correlation: Associazione monotonica con target
   - Mutual Information: Dipendenza non lineare
   - ANOVA F-test: Differenze tra medie di classe

2. **Embedded Method** (considera interazioni):
   - Random Forest Importance: Valuta feature durante training

3. **Consensus Aggregation**:
   - Ranking medio dei 4 metodi
   - Riduce overfitting su metodo singolo
   - Identifica feature robuste su criteri multipli

### **Anti-Leakage Policy:**
Tutte le statistiche (varianza, correlazione, importanza) calcolate **SOLO su X_train**, poi applicate a Val/Test.

### **5.1 Variance Threshold**

**Teoria:** Feature con varianza ≈ 0 sono **quasi-costanti** e non forniscono informazione discriminante.

**Implementazione:** Rimuoviamo feature con `Var(X) = 0.01` (quasi costanti).

In [ ]:
print("[4/5] Feature Selection Pipeline SOTA...\n")

# A. Variance Threshold
print("1. Variance Threshold...")
var_selector = VarianceThreshold(threshold=0.01)
var_selector.fit(X_train)
feat_var = X_train.columns[var_selector.get_support()]

X_train_v = X_train.loc[:, feat_var]
removed_var = X_train.shape[1] - len(feat_var)
print(f"    {removed_var} feature rimosse. Rimaste: {X_train_v.shape[1]}")

# Salva output
low_var_features = X_train.columns[~var_selector.get_support()].tolist()
with open(os.path.join(INTERMEDIATE_PATH, "06_low_variance_features.json"), 'w') as f:
    json.dump(low_var_features, f, indent=2)

### **5.2 Correlation Filter ≥ 0.95**

**Teoria:** Feature altamente correlate (|r| ≥ 0.95) sono **ridondanti** e causano:
1. **Multicollinearità**: Instabilità nei coefficienti del modello
2. **Overfitting**: Il modello memorizza rumore correlato

**Implementazione:**
- Calcolo matrice di correlazione di Pearson
- Identificazione coppie con |r| ≥ 0.95
- Rimozione di una feature per coppia (manteniamo quella con ranking migliore)

In [ ]:
# B. Correlation Filter
print("\n2. Correlation Filter ≥ 0.95...")
corr_matrix = X_train_v.corr().abs()
upper = corr_matrix.where(
    np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
)

# Trova coppie con correlazione alta
high_corr_pairs = []
for column in upper.columns:
    if any(upper[column] >= STATE_VARS['CORR_THRESHOLD']):
        for idx in upper.index[upper[column] >= STATE_VARS['CORR_THRESHOLD']]:
            high_corr_pairs.append({
                'Feature1': column,
                'Feature2': idx,
                'Correlation': upper.loc[idx, column],
            })

# Salva coppie multicollineari
if high_corr_pairs:
    pd.DataFrame(high_corr_pairs).to_csv(
        os.path.join(INTERMEDIATE_PATH, "09_multicollinearity_pairs.csv"),
        index=False,
    )

# Rimuovi feature ridondanti
to_drop_corr = [column for column in upper.columns if any(upper[column] >= STATE_VARS['CORR_THRESHOLD'])]
X_train_c = X_train_v.drop(columns=to_drop_corr)

print(f"    {len(to_drop_corr)} feature rimosse. Rimaste: {X_train_c.shape[1]}")

# Salva candidati alla rimozione
drop_candidates = {'multicollinearity_candidates': to_drop_corr}
with open(os.path.join(INTERMEDIATE_PATH, "10_multicollinearity_drop_candidates.json"), 'w') as f:
    json.dump(drop_candidates, f, indent=2)

### **5.3 Feature-Target Correlation (Spearman)**

**Teoria:** Correlazione di Spearman misura **associazione monotonica** (non necessariamente lineare) tra feature e target.

**Vantaggi vs Pearson:**
- Robusta a outlier
- Cattura relazioni non lineari monotone
- Non assume distribuzione normale

**Interpretazione:** rho ∈ [-1, 1]
- |ρ| ≈ 1: Forte associazione monotonica
- |ρ| ≈ 0: Nessuna associazione monotonica


In [ ]:
# C. Feature-Target Correlation (Spearman)
print("\n3. Feature-Target Correlation (Spearman)...")

# Encode target per correlazione
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)

corr_results = []
for col in X_train_c.columns:
    try:
        corr, p_val = spearmanr(X_train_c[col], y_train_encoded)
        corr_results.append({
            'Feature': col,
            'SpearmanCorr': abs(corr),
            'CorrSign': np.sign(corr),
            'pvalue': p_val,
        })
    except:
        corr_results.append({
            'Feature': col,
            'SpearmanCorr': np.nan,
            'CorrSign': np.nan,
            'pvalue': np.nan,
        })

corr_df = pd.DataFrame(corr_results).sort_values('SpearmanCorr', ascending=False)
corr_df.to_csv(os.path.join(INTERMEDIATE_PATH, "08_feature_target_correlation.csv"), index=False)

print(f"    Correlazioni calcolate per {len(corr_df)} feature")
print(f"   Top 5 feature correlate al target:")
print(corr_df.head()[['Feature', 'SpearmanCorr']].to_string(index=False))

### **5.4 Mutual Information**

**Teoria:** Mutual Information (MI) misura la **dipendenza statistica generale** tra feature e target, catturando anche relazioni **non lineari** e **non monotone**.

**Complementarità con Correlazione:**
- Correlazione: Solo relazioni lineari/monotone
- MI: Qualsiasi dipendenza statistica

**Esempio:** Feature con correlazione bassa ma MI alta → Relazione non lineare importante!

In [ ]:
# D. Mutual Information
print("\n4. Mutual Information (può richiedere qualche minuto)...")

mi_scores = mutual_info_classif(
    X_train_c, y_train_encoded,
    random_state=STATE_VARS['RANDOM_SEED'], n_jobs=-1,
)

mi_df = pd.DataFrame({
    'Feature': X_train_c.columns,
    'MI_Score': mi_scores,
}).sort_values('MI_Score', ascending=False)

mi_df.to_csv(os.path.join(INTERMEDIATE_PATH, "11_mutual_information_scores.csv"), index=False)

print(f"    MI calcolata per {len(mi_df)} feature")
print(f"    Top 5 feature per MI:")
print(mi_df.head()[['Feature', 'MI_Score']].to_string(index=False))

### **5.5 ANOVA F-test**

**Teoria:** ANOVA (Analysis of Variance) F-test valuta se le **medie di una feature differiscono significativamente tra le classi**.

**Utilità per questo dataset:**
CIC-IDS-2017 ha 15 classi (BENIGN + 14 attacchi) -> ridotte ad 8. ANOVA identifica feature che separano bene **tutte** le classi contemporaneamente.

**Assunzioni (tollerabili con grandi dataset):**
- Normalità
- Varianza simile

**p-value < 0.05:** Differenze statisticamente significative

In [ ]:
# E. ANOVA F-test
print("\n5. ANOVA F-test...")

f_scores, f_pvalues = f_classif(X_train_c, y_train_encoded)

anova_df = pd.DataFrame({
    'Feature': X_train_c.columns,
    'F_Statistic': f_scores,
    'pvalue': f_pvalues,
}).sort_values('F_Statistic', ascending=False)

anova_df.to_csv(os.path.join(INTERMEDIATE_PATH, "12_anova_f_statistics.csv"), index=False)

print(f"    ANOVA F-test per {len(anova_df)} feature")
print(f"    Top 5 feature per F-statistic:")
print(anova_df.head()[['Feature', 'F_Statistic']].to_string(index=False))

### **5.6 Random Forest Importance**

**Teoria:** Random Forest calcola **Feature Importance** come **Mean Decrease in Impurity (MDI)** durante il training.

**Come Funziona:**
1. RF costruisce N alberi
2. Ogni split su feature X riduce l'impurity (Gini o Entropy)
3. Importance(X) = Media delle riduzioni di impurity su tutti gli alberi

**Vantaggi:**
- Cattura **interazioni non lineari** tra feature
- Robusto a scale diverse (normalizzazione non richiesta)

**Bias Noto:**
RF tende a favorire feature con alta cardinalità. Per questo usiamo Consensus con altri metodi.

**`class_weight='balanced'`:** Compensa lo sbilanciamento di CIC-IDS-2017 pesando inversamente alla frequenza di classe.

In [ ]:
print("\n6. RF Importance (Training...)")

rf = RandomForestClassifier(
    n_estimators=STATE_VARS['RF_N_ESTIMATORS'],
    random_state=STATE_VARS['RANDOM_SEED'],
    class_weight='balanced',
    n_jobs=-1,
    max_depth=20,
)
rf.fit(X_train_c, y_train_encoded)

rf_importance = pd.DataFrame({
    'Feature': X_train_c.columns,
    'Importance': rf.feature_importances_,
}).sort_values('Importance', ascending=False)

rf_importance.to_csv(os.path.join(INTERMEDIATE_PATH, "13_rf_feature_importance.csv"), index=False)
print(f"    {len(rf_importance)} feature analizzate")
print(f"    Top 5: {rf_importance.head(5)['Feature'].tolist()}")

plt.figure(figsize=(12, 8))
top_20_rf = rf_importance.head(20)
plt.barh(range(len(top_20_rf)), top_20_rf['Importance'], color='steelblue', alpha=0.8)
plt.yticks(range(len(top_20_rf)), top_20_rf['Feature'])
plt.xlabel('Importanza (Mean Decrease in Impurity)', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.title('Top 20 Feature - Random Forest Importance', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(IMG_PATH, "07_rf_feature_importance.png"), dpi=150, bbox_inches='tight')
plt.show()




### **5.7 Consensus Ranking & Selezione Finale**

**Teoria:** **Ensemble Feature Selection** aggrega ranking di metodi multipli per identificare feature **robuste** su criteri diversi.

**Problema dei Metodi Singoli:**
- Correlation: Bias verso relazioni lineari
- MI: Può overfit con pochi campioni per valore discreto
- ANOVA: Sensibile a outlier e non-normalità
- RF: Bias verso alta cardinalità

**Soluzione SOTA (Tripathi et al., 2024):**

### **Consensus Ranking Formula:**
Per ogni feature X:
1. Rank_Corr(X): Posizione nel ranking Spearman
2. Rank_MI(X): Posizione nel ranking MI
3. Rank_ANOVA(X): Posizione nel ranking F-statistic
4. Rank_RF(X): Posizione nel ranking Importance

**Average_Rank(X) = (Rank_Corr + Rank_MI + Rank_ANOVA + Rank_RF) / 4**

### **Selezione Top-20:**
Selezioniamo le **20 feature con Average_Rank più basso** (ranking migliore).

**Vantaggi:**
1. **Robustezza**: Feature che performano bene su criteri multipli
2. **Diversità**: Combina prospettive lineari, non lineari, embedded
3. **Riduzione Overfitting**: Evita bias di metodo singolo

In [ ]:
# G. Consensus Ranking
print("\n7. Consensus Ranking...")

# Normalizza ranking (1 = migliore)
corr_df['Corr_Rank'] = corr_df['SpearmanCorr'].rank(ascending=False)
mi_df['MI_Rank'] = mi_df['MI_Score'].rank(ascending=False)
anova_df['ANOVA_Rank'] = anova_df['F_Statistic'].rank(ascending=False)
rf_importance['RF_Rank'] = rf_importance['Importance'].rank(ascending=False)

# Merge tutti i ranking
consensus = pd.DataFrame({'Feature': X_train_c.columns})
consensus = consensus.merge(corr_df[['Feature', 'Corr_Rank']], on='Feature', how='left')
consensus = consensus.merge(mi_df[['Feature', 'MI_Rank']], on='Feature', how='left')
consensus = consensus.merge(anova_df[['Feature', 'ANOVA_Rank']], on='Feature', how='left')
consensus = consensus.merge(rf_importance[['Feature', 'RF_Rank']], on='Feature', how='left')

# Calcola ranking medio
consensus['Average_Rank'] = consensus[['Corr_Rank', 'MI_Rank', 'ANOVA_Rank', 'RF_Rank']].mean(axis=1)
consensus = consensus.sort_values('Average_Rank')

consensus.to_csv(os.path.join(INTERMEDIATE_PATH, "14_consensus_ranking.csv"), index=False)

print(f"    Consensus Ranking calcolato per {len(consensus)} feature")
print(f"    Top 10 feature per consenso:")
print(consensus.head(10)[['Feature', 'Average_Rank']].to_string(index=False))

# --- SELEZIONE TOP-N FEATURES ---
print(f"\n Selezione Top-{STATE_VARS['N_FEATURES_FINAL']} feature dal Consensus Ranking...")
top_n_features = consensus.head(STATE_VARS['N_FEATURES_FINAL'])['Feature'].tolist()

X_train_final = X_train_c.loc[:, top_n_features]

print(f"    Feature Finali Selezionate: {len(top_n_features)}")
print(f"\n Lista Feature Finali:")
for i, feat in enumerate(top_n_features, 1):
    print(f"   {i:2d}. {feat}")


# Plot Consensus Ranking Comparison
fig, ax = plt.subplots(figsize=(14, 8))
top_20_consensus = consensus.head(20)

x = np.arange(len(top_20_consensus))
width = 0.2

ax.barh(x - 1.5*width, top_20_consensus['Corr_Rank'], width, label='Correlation', alpha=0.8)
ax.barh(x - 0.5*width, top_20_consensus['MI_Rank'], width, label='Mutual Information', alpha=0.8)
ax.barh(x + 0.5*width, top_20_consensus['ANOVA_Rank'], width, label='ANOVA F-test', alpha=0.8)
ax.barh(x + 1.5*width, top_20_consensus['RF_Rank'], width, label='Random Forest', alpha=0.8)

ax.set_yticks(x)
ax.set_yticklabels(top_20_consensus['Feature'])
ax.set_xlabel('Ranking Position (lower = better)', fontsize=12)
ax.set_ylabel('Feature', fontsize=12)
ax.set_title('Consensus Ranking - Top 20 Features (Comparison dei 4 Metodi)', fontsize=14, fontweight='bold')
ax.legend(loc='lower right')
ax.invert_yaxis()
ax.invert_xaxis()  # Ranking più basso (migliore) a sinistra
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(IMG_PATH, "08_consensus_ranking_comparison.png"), dpi=150, bbox_inches='tight')
plt.show()
print(f"    Grafico salvato: {os.path.join(IMG_PATH, '08_consensus_ranking_comparison.png')}")


## **6. Applicazione Trasformazioni a Val/Test**

### **Anti-Leakage Policy**

Applichiamo le **stesse trasformazioni** (rimozione feature) a Val e Test usando le maschere calcolate su Train.

**CRITICAL:** Val e Test NON hanno influenzato la Feature Selection. Questo garantisce:
1. **No Data Leakage**: Statistiche calcolate solo su Train
2. **Valutazione Onesta**: Val/Test simulano dati mai visti
3. **Generalizzazione Reale**: Performance su Val/Test riflette deployment

**Verifica Coerenza:** Train, Val, Test devono avere la stessa shape in colonne.

In [ ]:
print("[5/5] Applicazione trasformazioni a Val/Test...")

# Applica stesse maschere a Val e Test
X_val_final = X_val.loc[:, top_n_features]
X_test_final = X_test.loc[:, top_n_features]

print(f"\n Dataset Finali:")
print(f"   Train: {X_train_final.shape}")
print(f"   Val:   {X_val_final.shape}")
print(f"   Test:  {X_test_final.shape}")

# Verifica coerenza
assert list(X_train_final.columns) == list(X_val_final.columns) == list(X_test_final.columns), \
    " ERRORE: Le colonne di Train/Val/Test non coincidono!"

print("\n Verifica coerenza: OK - Tutte le colonne coincidono")

# Heatmap correlazione tra le feature finali selezionate
plt.figure(figsize=(14, 12))
corr_final = X_train_final.corr()
mask = np.triu(np.ones_like(corr_final, dtype=bool))
sns.heatmap(corr_final, mask=mask, annot=False, cmap='RdBu_r', center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title(f'Correlazione tra le {len(top_n_features)} Feature Finali Selezionate',
          fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.savefig(os.path.join(IMG_PATH, "09_final_features_correlation_heatmap.png"), dpi=150, bbox_inches='tight')
plt.show()
print(f"    Grafico salvato: {os.path.join(IMG_PATH, '09_final_features_correlation_heatmap.png')}")


## **7. Salvataggio Dataset Processati**

Salviamo i dataset finali in formato **Parquet** (compresso, efficiente) per il prossimo notebook.

**Output:**
- X_train.parquet, y_train.parquet
- X_val.parquet, y_val.parquet
- X_test.parquet, y_test.parquet
- 20_selected_features.json (lista feature finali)

In [ ]:
print("\n Salvataggio dataset processati...")

# Salva Train
X_train_final.to_parquet(
    os.path.join(OUTPUT_PATH, "X_train.parquet"),
    index=False,
    compression='snappy'
)
y_train.to_frame().to_parquet(
    os.path.join(OUTPUT_PATH, "y_train.parquet"),
    index=False,
    compression='snappy'
)

# Salva Val
X_val_final.to_parquet(
    os.path.join(OUTPUT_PATH, "X_val.parquet"),
    index=False,
    compression='snappy'
)
y_val.to_frame().to_parquet(
    os.path.join(OUTPUT_PATH, "y_val.parquet"),
    index=False,
    compression='snappy'
)

# Salva Test
X_test_final.to_parquet(
    os.path.join(OUTPUT_PATH, "X_test.parquet"),
    index=False,
    compression='snappy'
)
y_test.to_frame().to_parquet(
    os.path.join(OUTPUT_PATH, "y_test.parquet"),
    index=False,
    compression='snappy'
)

# Salva lista feature finali
final_features = {'selected_features': top_n_features}
with open(os.path.join(INTERMEDIATE_PATH, "15_selected_features.json"), 'w') as f:
    json.dump(final_features, f, indent=2)

print("\n COMPLETATO! Dataset salvati in:")
print(f"   {OUTPUT_PATH}")
print(f"\n Feature finali: {len(top_n_features)}")
print(f"   Riduzione: {X_train.shape[1]} → {len(top_n_features)} ({100*(1-len(top_n_features)/X_train.shape[1]):.1f}% riduzione)")

## **8. Report Riepilogativo**

In [ ]:
print("\n" + "="*60)
print(" REPORT FINALE - FEATURE ENGINEERING & SELECTION")
print("="*60)
print(f"\n🔹 Feature Iniziali (post-sanitizzazione): {X_train.shape[1]}")
print(f"🔹 Dopo Variance Threshold: {X_train_v.shape[1]} (-{removed_var})")
print(f"🔹 Dopo Correlation Filter (≥0.95): {X_train_c.shape[1]} (-{len(to_drop_corr)})")
print(f"🔹 Dopo Consensus Ranking (Top-{STATE_VARS['N_FEATURES_FINAL']}): {len(top_n_features)}")
print(f"\n Riduzione Totale: {100*(1-len(top_n_features)/X_train.shape[1]):.1f}%")
print(f"\n  Output salvati in:")
print(f"   - Dataset processati: {OUTPUT_PATH}")
print(f"   - Analisi intermedie: {INTERMEDIATE_PATH}")
print(f"\n  Feature Finali Selezionate ({len(top_n_features)}):")
for i, feat in enumerate(top_n_features, 1):
    avg_rank = consensus[consensus['Feature'] == feat]['Average_Rank'].values[0]
    print(f"   {i:2d}. {feat:<30s} (Avg Rank: {avg_rank:.2f})")
print("\n" + "="*60)
print(" Pronto per il Notebook 03: Model Training & Evaluation")
print("="*60)